In [1]:
import sys, os
# Append ../src to the system path
sys.path.append("../src")

import config
from utils import *

## Set up a retry helper to handle per-minute quota limits during "Run all."

In [2]:
from google.api_core import retry
from google import genai
from google.genai import types

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

GOOGLE_API_KEY = config.GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)

# Generate recipe from text

In [3]:
response = generate_recipe(["chicken", "rice", "broccoli"])

In [4]:
print(response)

{'title': 'Ginger-Glazed Chicken with Sesame Broccoli and Coconut Rice', 'cooking_time': '45 minutes', 'ingredients': ['Chicken breasts: 2, boneless and skinless', 'Rice: 1 cup, basmati or jasmine', 'Broccoli florets: 1 large head', 'Soy sauce: 1/4 cup', 'Honey: 2 tablespoons', 'Fresh ginger: 1 tablespoon, grated', 'Garlic: 2 cloves, minced', 'Sesame oil: 1 tablespoon', 'Sesame seeds: 1 tablespoon', 'Coconut milk: 1 cup', 'Water: 1 cup', 'Green onions: 2, thinly sliced (for garnish)', 'Cornstarch: 1 tablespoon', 'Vegetable oil: 2 tablespoons'], 'instructions': ['Prepare the coconut rice: In a saucepan, combine rice, coconut milk, and water. Bring to a boil, then reduce heat to low, cover, and simmer for 18-20 minutes, or until liquid is absorbed and rice is cooked.', 'Make the ginger-garlic glaze: In a small bowl, whisk together soy sauce, honey, grated ginger, minced garlic, and cornstarch. Set aside.', 'Prepare the broccoli: Cut broccoli florets into bite-sized pieces. Toss with 1 ta

# Generate recipe from Image

In [5]:
image_path=os.path.join(config.ROOT_PATH, "data/ingredients.jpg")
raw_ingredients=detect_ingredients(image_path)
response = generate_recipe(raw_ingredients)

print(response)

{'title': 'Coriander-Lime Chicken Skewers with Cumin-Ginger Marinade', 'cooking_time': '30 minutes (plus 2 hours marinating)', 'ingredients': ['Chicken breast: 500g, cut into 1-inch cubes', 'Fresh coriander: 1 bunch, roughly chopped', 'Lemon juice: 3 tablespoons', 'Green chilli: 1-2, finely chopped (adjust to taste)', 'Salt: 1 teaspoon (or to taste)', 'Cumin powder: 2 teaspoons', 'Fresh ginger: 1 tablespoon, grated', 'Olive oil: 2 tablespoons', 'Wooden skewers: 8-10, soaked in water for 30 minutes'], 'instructions': ['In a bowl, combine coriander, lemon juice, green chilli, salt, cumin powder, grated ginger, and olive oil. Mix well to create a marinade.', 'Add the chicken cubes to the marinade, ensuring they are well coated. Cover the bowl and refrigerate for at least 2 hours, or preferably overnight.', 'Preheat your grill or oven to medium-high heat (about 200°C/400°F).', 'Thread the marinated chicken cubes onto the soaked wooden skewers.', 'Grill or bake the skewers for 15-20 minutes

Here are the instructions for the dish.

In [6]:

response["instructions"]

['In a bowl, combine coriander, lemon juice, green chilli, salt, cumin powder, grated ginger, and olive oil. Mix well to create a marinade.',
 'Add the chicken cubes to the marinade, ensuring they are well coated. Cover the bowl and refrigerate for at least 2 hours, or preferably overnight.',
 'Preheat your grill or oven to medium-high heat (about 200°C/400°F).',
 'Thread the marinated chicken cubes onto the soaked wooden skewers.',
 'Grill or bake the skewers for 15-20 minutes, turning occasionally, until the chicken is cooked through and lightly browned. Ensure the internal temperature reaches 74°C/165°F.',
 'Serve immediately.']

In [7]:
os.mkdir(response["title"])

In [8]:
from io import BytesIO

images = []
for step in response["instructions"]:
  image_response = client.models.generate_content(
    model="gemini-2.0-flash-exp-image-generation",
    contents=f"Generate an image for this step: {step}",
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE'],
     # system_instruction="You are a helpful assistant that generates images for cooking instructions. You will help me create images for each step of the recipe.",
    ))

  for part in image_response.candidates[0].content.parts:
    if part.text is not None:
      print(part.text)
    elif part.inline_data is not None:
      image = Image.open(BytesIO((part.inline_data.data)))
      # image.save(response["title"] + "/" + step + ".jpg")
      images.append(image)
      # print("Image saved for step:", step)